In [ ]:
## Fine-tunning bert cased for Resume Screening.

In [ ]:
!pip install evaluate              #for model evaluation

In [ ]:
# !pip install wandb                       #for plotting training and result metrics

In [ ]:
#import libraries

from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import TrainingArguments,Trainer
import evaluate
import numpy as np



In [ ]:
import pandas as pd
from datasets import Dataset

In [ ]:
#load the data
raw_dataset=load_dataset("csv",data_files="/kaggle/input/ai-resume-screening/AI_Resume_Screening.csv")
raw_dataset=raw_dataset['train']

### **Preprocessing**

In [ ]:
raw_dataset

In [ ]:
raw_dataset.features

In [ ]:
panddata=raw_dataset.to_pandas()    #for mergeing the text columns 

In [ ]:
panddata

In [ ]:
#create class labels column in numeric using map
panddata['labels']=panddata['Recruiter Decision'].map({'Reject':0,'Hire':1})

In [ ]:
#merge the text columns
panddata['Resume_details']=panddata[['Skills', 'Experience (Years)', 'Education',
       'Certifications', 'Job Role','Salary Expectation ($)', 'Projects Count', 'AI Score (0-100)']].astype(str).agg(','.join,axis=1)

In [ ]:
panddata    #new table with class labels(num) and merged column

In [ ]:
#convert dataset from pandas dataframe to Hugging Face dataset
final_raw_data=Dataset.from_pandas(panddata)

In [ ]:
#train test validation split
final_raw_data=final_raw_data.shuffle(seed=14)
finaldata=final_raw_data.class_encode_column('labels').train_test_split(test_size=0.25,seed=12,stratify_by_column='labels')  #encode numeric column to class column to stratify
valtestdata=finaldata['test']
valtest=valtestdata.train_test_split(test_size=0.5,seed=12)
val=valtest['train']
test=valtest['test']

In [ ]:
#Tokenizer initialization
checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [ ]:
train=finaldata['train']

In [ ]:
#Function that returns the tokenized values called by the map function
def tokenize_function(Resume):
  return tokenizer(Resume['Resume_details'],truncation=True)

In [ ]:
#applying map function to tokenize each data one at time
tokenized_train=train.map(tokenize_function)

In [ ]:
tokenized_train

In [ ]:
tokenized_val=val.map(tokenize_function)

In [ ]:
tokenized_test=test.map(tokenize_function)

In [ ]:
#For Dynamic Padding
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

### **Hyperparameters**

In [ ]:
trainingargs=TrainingArguments('test_trainer',num_train_epochs=5,learning_rate=2e-05,weight_decay=0.01,report_to='none',eval_strategy='epoch',)

In [ ]:
trainingargs

In [ ]:
#to use wandb plotting
# import wandb
# wandb.login(key='apikey') #creating a free account provides api key
# wandb.init(project_name)
#in trainingargs report_to parameter

### **Model**

In [ ]:
#initialize model for 2 class labels
model=AutoModelForSequenceClassification.from_pretrained(checkpoint,num_labels=2)

In [ ]:
#metric function given to trainer api to evaluate validation 
def compute_metrics(output):
    prediction,label=output
    pred=np.argmax(output.predictions,axis=-1)
    metrics=evaluate.combine(['accuracy','precision','recall','f1'])
    return metrics.compute(predictions=pred,references=label)
    

### **Trainer API**

In [ ]:
trainer=Trainer(
    model,
    trainingargs,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    data_collator=data_collator,
    processing_class=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()  #train the model using initialized trainer

### **Testing**

In [ ]:
output=trainer.predict(tokenized_test)     #output prediction for test data


In [ ]:
pred=np.argmax(output.predictions,axis=-1)   #predicted output
pred

In [ ]:
output.label_ids   #actual output

In [ ]:
#evaluation on test data
metrics=evaluate.combine(['accuracy','precision','recall','f1'])
metrics.compute(predictions=pred,references=output.label_ids)

In [ ]:
#confusion matrix
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Generate confusion matrix
cm = confusion_matrix(pred, output.label_ids)

# Plot with seaborn
plt.figure(figsize=(6,4))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Reject', 'Hire'],
            yticklabels=['Reject', 'Hire'])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()